In [ ]:
from Crypto.Cipher import AES
from Crypto import Random
import base64
import binascii
import numpy as np
import pandas as pd
from IPython.display import HTML
import json

# Web Scrapping
import requests as rq
from bs4 import BeautifulSoup as bfs

In [ ]:
HTML('<iframe src=https://www.datasport.com/sys/myds/ajax/getDSInfo.htm?acode=5CHP784HD width=1000 height=350></iframe>')

https://www.datasport.com/sys/myds/ajax/getDSInfo.htm?acode=5CHP784HD

JS: 
```
var key = CryptoJS.enc.Base64.parse("AYf833rD2Q1oM0r6A7h+/g==");
var iv  = CryptoJS.enc.Base64.parse("M6OUM+Td58TdudRQK4kF1A==");
var decrypted = CryptoJS.AES.decrypt(resultEnc, key, {iv: iv});
var result = JSON.parse(decrypted.toString(CryptoJS.enc.Utf8));
```

Results:
```
resultEnc = '3zxaer+64X4MlhwuD6brwEHBNFvFQ1/1v6HvtgY6H8cXvrxxGlWtGdWGRqfdmh5BmOnTTj0AplYL4GXExOqQU8s2quJTqvawdfazWRNL/9y76ldSjVbEl3XaFhl7yzA3uX36uOswsfUEXvQrvXeSWd8+KaVlbzyUDZnlytuBi+GWaipKz6NdSqmY8U3nPdaoVDf7euSP/7Z3L3sLLYLmQ0UgOtp+2+GSDL8cVHNMZTHsBXwP7LOQtXKjOLBJ0wm7svr+omqmh1Hr8kG2Kcj2JTkUQd+mrvbTeofzmzYLWezHJzwMeYMXsx5VdgbJtnaJoi5Jv6MVVJkeQFOJ8t1hYfuQoFfpIrDZLS4pdY8FhQFFOuPELOZuDeAAXwAG8Y2idzpaZIPD1V+4+iisal0Ud0Sl6XyjTqSaz4L9ceYJMKAdo5/kXZWqiBotN/u+rUdmtAaVAoMVA7zhPN7EIvpN+dWfLn/37ukWY1R51m99C5ryH0JoZSA2DmFpziEfqLk6y6jvGEY/EVx3sRiFgh1VsYs+o3VDKlYEsKh5sPoGi9pmQiZOTdGazXFxbCpVJJT3KxU12bleNrQdrABVvxAhh9GgtfWLFdozlpW11K8TbhTheu6Qr8yRBnJpq5gKe5ti'

key = {
    sigBytes: 16,
    words: {
        0: 25689311,
        1: 2059655437,
        2: 1748191994,
        3: 62422782
    }
}

iv = {
    sigBytes: 16,
    words: {
        0: 866358323,
        1: -455219260,
        2: -575024048,
        3: 730400212
    }
}

result = {
    catRank: 43,
    categoryName: "STC Olympic Hommes CA 35-44",
    entryArt: "Bezahlt",
    entryPayart: "Online",
    eventDate: "11.09.2016",
    eventName: "Triathlon Yverdon-les-Bains",
    eventRaceNr: "B628CCC3D9A7DDBF558DA920B9E33C7D"
    overCategoryName: "STC Olympic Hommes Overall",
    overRank: 122,
    provider: "Datasport",
    raceNr: 18594,
    racePayload: "E09137F006F1390053225B57ABFA512DF7BC65269C53D92AC9AB3E2E044E6DFD",
    resultState: "Klassiert",
    runtime: "3:02.32,0",
    startNumber: 166
}
```

In [ ]:
def decrypt_response(response):
    
    html_page = '<p id="crypted" style="display:none;">' + response + '</p>'
    
    jquery_file = open('Lib/jquery.min.js', 'r')
    crypto_js_file = open('Lib/cryptojs-aes.js', 'r')
    
    javascript = '<script type="text/Javascript">' + jquery_file.read() + '</script>'
    javascript += '<script type="text/Javascript">' + crypto_js_file.read() + '</script>'
    
    javascript += """
        <script type="text/Javascript">
            var key = CryptoJS.enc.Base64.parse("AYf833rD2Q1oM0r6A7h+/g==");
            var iv  = CryptoJS.enc.Base64.parse("M6OUM+Td58TdudRQK4kF1A==");

            var crypted = $('#crypted').text();

            var decrypted = CryptoJS.AES.decrypt(crypted, key, {iv: iv});
            var result = decrypted.toString(CryptoJS.enc.Utf8);

            var command = "result = '" + result + "'";

            var kernel = IPython.notebook.kernel;
            kernel.execute(command);

        </script>
    """

    HTML(html_page + javascript)
    return json.loads(result)

In [ ]:
resultEnc = '3zxaer+64X4MlhwuD6brwEHBNFvFQ1/1v6HvtgY6H8cXvrxxGlWtGdWGRqfdmh5BmOnTTj0AplYL4GXExOqQU8s2quJTqvawdfazWRNL/9y76ldSjVbEl3XaFhl7yzA3uX36uOswsfUEXvQrvXeSWd8+KaVlbzyUDZnlytuBi+GWaipKz6NdSqmY8U3nPdaoVDf7euSP/7Z3L3sLLYLmQ0UgOtp+2+GSDL8cVHNMZTHsBXwP7LOQtXKjOLBJ0wm7svr+omqmh1Hr8kG2Kcj2JTkUQd+mrvbTeofzmzYLWezHJzwMeYMXsx5VdgbJtnaJoi5Jv6MVVJkeQFOJ8t1hYfuQoFfpIrDZLS4pdY8FhQFFOuPELOZuDeAAXwAG8Y2idzpaZIPD1V+4+iisal0Ud0Sl6XyjTqSaz4L9ceYJMKAdo5/kXZWqiBotN/u+rUdmtAaVAoMVA7zhPN7EIvpN+dWfLn/37ukWY1R51m99C5ryH0JoZSA2DmFpziEfqLk6y6jvGEY/EVx3sRiFgh1VsYs+o3VDKlYEsKh5sPoGi9pmQiZOTdGazXFxbCpVJJT3KxU12bleNrQdrABVvxAhh9GgtfWLFdozlpW11K8TbhTheu6Qr8yRBnJpq5gKe5ti'
decrypt_response(resultEnc)

Get all data about runners.

In [ ]:
ADRESS_PERSONNAL_PAGE = 'https://www.datasport.com/sys/myds/ajax/getDSInfo.htm?acode='
FILE_NAME = 'unique_acode_2009_2015.csv'
PATH_TO_DATA = './Data/acode_2009_2015/'

data_acode = pd.read_csv(PATH_TO_DATA+FILE_NAME, sep='\t')
data_acode.head()

In [ ]:
for acode in data_acode['acode']:
    url = 'https://www.datasport.com/sys/myds/ajax/getDSInfo.htm?acode='+acode
    print(url)
    filters_page = rq.get(url) 
    page = bfs(filters_page.text, 'html.parser')
    
    print(page)
    
    # get the birtyear
    matches = page.findAll("div", { "class" : "small" })

    # get information on the table.
    
    break;

In [ ]:
year.split(":")[1:][0]